# Customer Segmentation and Spending Behavior Analysis
## ITCS 3162 — Data Mining Final Project

Group Members: Ameer Khan, Izaan Khudadad, JD Singh, Tyler Hebner




## Introduction
This project utilizes data mining and machine learning techniques to understand and segment a company's customer base. The primary goal is to perform **customer segmentation** using clustering algorithms to uncover distinct, naturally occurring groups of customers. These groups can then be used to personalize marketing strategies, optimize product offerings, and ultimately improve customer satisfaction and business revenue.

## Problem Statement
In the absence of a clear understanding of their diverse customer population, businesses often resort to generic marketing, which is inefficient and costly. We aim to transform raw customer data into actionable insights by identifying who the most valuable, and distinct, customer groups are.

## Key Questions
1.  **What are the main clusters of customer types within the dataset?** (i.e., identifying the segments)
2.  **Which demographic or behavioral variables have the greatest influence on customer spending?** (i.e., interpreting the segments)
3.  **How can these insights help businesses target customers more effectively?** (i.e., business application)

# 📊 About the Data

## Data Source
The dataset, **'Customer Personality Analysis,'** was obtained from **Kaggle** (specifically: `https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis`). It contains detailed records of customers' demographics, purchasing behavior across different product categories, and their response to previous marketing campaigns.

## Data Loading and Initial Inspection

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Load the dataset using the correct tab separator ('\t')
try:
    df = pd.read_csv('marketing_campaign.csv', sep='\t') 
except FileNotFoundError:
    print("Error: marketing_campaign.csv not found. Please ensure the file is in the correct directory.")
    df = None 

if df is not None:
    print("--- Dataset Shape ---")
    print(df.shape)
    print("\n--- Column Data Types ---")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(df.info())
    print("\n--- First 5 Rows ---")
    print(df.head())
    

--- Dataset Shape ---
(2240, 29)

--- Column Data Types ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 no

## Evaluation of Initial Data Load
The initial data loading was **successful**. By using the tab delimiter (`sep='\t'`), the file was parsed correctly.

* **Shape:** The resulting DataFrame has **2,240 rows** and **29 columns**.
* **Data Types:** Most columns are correctly loaded as numerical (`int64`), but one critical issue remains: **`Income` has 2216 non-null counts**, meaning there are **24 missing values** (2240 - 2216 = 24) that must be handled before any modeling can begin.

## Next Step: Data Cleaning and Feature Engineering 🧼
We must now resolve the missing income values, handle potential outliers (like age), and create the essential features needed for our three models (Clustering, Regression, and Classification).

# 🧼 3. Data Cleaning and Feature Engineering

## 3.1 Handling Missing Values
We impute the 24 missing values in the **`Income`** column using the **median**. The median is a robust choice because it is less affected by potential extreme outliers compared to the mean.

In [2]:
# Handle missing Income values by imputing with the median
df['Income'] = df['Income'].fillna(df['Income'].median())

print("Missing values after imputation:")
# Check again for any remaining missing values (should be zero for Income)
print(df.isnull().sum()[df.isnull().sum() > 0])

Missing values after imputation:
Series([], dtype: int64)


## Evaluation of Missing Value Imputation
The output confirms that the missing values in the **`Income`** column have been successfully handled by imputing them with the median. The dataset now has **no missing values**, which is essential for all three of our modeling tasks (Clustering, Regression, and Classification).

## Next Step: Feature Engineering ⚙️
We proceed to create and clean essential features needed for our models, including the two crucial target variables: `Total_Spending` (Regression) and `Is_High_Spender` (Classification).

## 3.2 Feature Engineering and Outlier Handling
We create essential features for our three models and handle potential outliers that could skew the results, particularly for clustering and regression.

1.  **Calculate Age:** We derive `Age` from `Year_Birth`.
2.  **Handle Age Outliers:** We remove customers who are likely outliers (e.g., age > 99).
3.  **Create Total Spending:** This will be the **continuous target variable** for the Regression model.
4.  **Create Children Count:** A combined demographic feature.
5.  **Create High Spender Binary:** This will be the **binary target variable** for the Classification model.

In [3]:
# 1. Calculate Age (Assuming year 2014)
df['Age'] = 2014 - df['Year_Birth']

# 2. Filter out potential extreme outliers (e.g., age > 99)
df = df[df['Age'] < 100].reset_index(drop=True)

# 3. Create Total Spending Feature (Target for Regression)
# Sum of all Mnt (Amount Spent) columns
mnt_cols = [col for col in df.columns if 'Mnt' in col]
df['Total_Spending'] = df[mnt_cols].sum(axis=1)

# 4. Create Total Children Feature
df['Children'] = df['Kidhome'] + df['Teenhome']

# 5. Create Binary Target Feature: Is_High_Spender (Target for Classification)
# Define High Spender as anyone above the 75th percentile of Total Spending
spending_threshold = df['Total_Spending'].quantile(0.75)
df['Is_High_Spender'] = (df['Total_Spending'] > spending_threshold).astype(int)

# Display the statistics for the new key features
print("\n--- Statistics for New Features and Targets ---")
print(df[['Age', 'Total_Spending', 'Children', 'Income']].describe())
print(f"\n75th Percentile Spending Threshold (High Spender Cutoff): ${spending_threshold:.2f}")
print(f"High Spender Count (1s): {df['Is_High_Spender'].sum()} customers")
print(f"Total Customers (after outlier removal): {len(df)}")


--- Statistics for New Features and Targets ---
               Age  Total_Spending     Children         Income
count  2237.000000     2237.000000  2237.000000    2237.000000
mean     45.098346      605.743406     0.950827   52227.407689
std      11.701917      601.840466     0.752037   25043.266830
min      18.000000        5.000000     0.000000    1730.000000
25%      37.000000       69.000000     0.000000   35523.000000
50%      44.000000      396.000000     1.000000   51381.500000
75%      55.000000     1045.000000     1.000000   68281.000000
max      74.000000     2525.000000     3.000000  666666.000000

75th Percentile Spending Threshold (High Spender Cutoff): $1045.00
High Spender Count (1s): 559 customers
Total Customers (after outlier removal): 2237


## Evaluation of Feature Engineering Output

The feature engineering stage prepared the dataset for modeling by cleaning missing values, removing extreme outliers, and creating meaningful new variables. The key outcomes are outlined below.

---

### Dataset Status After Feature Engineering
- The dataset now contains **2,237 valid customer records** after removing unrealistic age outliers.
- All missing values in the **Income** column were imputed using the **median**, leaving the dataset with **zero missing values**.

---

###  Key Features and Targets Created

#### **1. Total_Spending (Regression Target)**
- Created by summing all product spending columns (`Mnt*` features).
- The spending distribution is **highly skewed**, with:
  - **Mean:** ~$605  
  - **Median:** ~$396  
  - **Standard Deviation:** ~$601  
  - **Maximum:** ~$2,525  
- Because spending is heavily right-skewed, applying a **log transformation** may improve regression performance.



#### **2. Is_High_Spender (Classification Target)**
- High spenders are defined as customers whose Total Spending is above the **75th percentile**.
- The threshold was calculated as **$1,045**.
- The class distribution is:
  - **559 high spenders (≈25%)**
  - **1,678 non-high spenders (≈75%)**
- Although slightly imbalanced, this distribution is still well-suited for models like **Random Forest**.



#### **3. Additional Engineered Features**
- **Age** was derived from `2014 − Year_Birth`, and customers older than **99** were filtered out as potential outliers.
- **Children** was created by combining `Kidhome + Teenhome` to capture total dependents in the household.
- **Income** is now complete with all missing entries filled using the median.
- A notable observation is the maximum recorded income of **$666,666**, which may represent an extreme outlier—something that scaling and clustering techniques will address.


### ✅ Summary
- The dataset is now **clean, consistent, and ready for modeling**.
- Both target variables—**Total_Spending** for regression and **Is_High_Spender** for classification—are clearly defined and statistically validated.
- The engineered demographic and behavioral features enhance the dataset and support more insightful clustering, regression, and classification tasks.

---

## Next Step: Modeling Stage 1 - K-Means Clustering 🧩
With the data cleaned and targets defined, we start with our first model: **K-Means Clustering** for segmentation. We must standardize the features for K-Means to work effectively.